In [24]:
import pandas as pd

## 날짜 기준 12개씩 묶어서 처리

In [2]:
# df = pd.read_csv('data/5528_spec.csv')
# df['created_at_datetime'] = pd.to_datetime(df['created_at'], unit='s', utc=True).dt.tz_convert('Asia/Seoul')

# # Extract the date part only
# df['date'] = df['created_at_datetime'].dt.date

# # Group by date and count occurrences
# date_counts = df.groupby('date').size()

# # Filter dates with fewer than 12 occurrences
# dates_with_less_than_12 = date_counts[date_counts != 12]

# print(dates_with_less_than_12)

# # 날짜에서 12개 미만이거나 초과되는 데이터를 추출하여 삭제
# dates_to_drop = dates_with_less_than_12.index

# # df에서 해당 날짜들에 해당하는 행을 제거
# df_cleaned = df[~df['created_at_datetime'].dt.date.isin(dates_to_drop)]

# # 파일로 저장
# df_cleaned.to_csv('5528_droped_data.csv', index=False)

### 12일씩 날짜로 묶인 csv 파일 로드

In [19]:
df = pd.read_csv('data/5528_droped_data.csv')

### 각 열의 문자열을 쉼표로 구분하여 float list로 변환

In [5]:
df['spectrum_x_amp'] = df['spectrum_x_amp'].apply(lambda x: [float(i) for i in x.split(',')])
df['spectrum_y_amp'] = df['spectrum_y_amp'].apply(lambda x: [float(i) for i in x.split(',')])
df['spectrum_z_amp'] = df['spectrum_z_amp'].apply(lambda x: [float(i) for i in x.split(',')])

### 인덱스 기준으로 12개씩 그룹화

In [6]:
grouped_df = df.groupby(df.index // 12).apply(lambda x: x.reset_index(drop=True))

In [7]:
normal_df = grouped_df[grouped_df['imbalance_health'] == 1]
error_df = grouped_df[grouped_df['imbalance_health'] != 1]

In [9]:
# normal_df = normal_df.drop(columns=['date', 'asset_id','time','imbalance_health', 'misalignment_health', 'looseness_health', 'bearing_health'])
normal_df = normal_df.drop(columns=['date', 'asset_id','time','misalignment_health', 'looseness_health', 'bearing_health'])

In [10]:
# error_df = error_df.drop(columns=['date','asset_id','time','imbalance_health','misalignment_health', 'looseness_health', 'bearing_health'])
error_df = error_df.drop(columns=['date','asset_id','time','misalignment_health', 'looseness_health', 'bearing_health'])

In [12]:
train_normal = pd.concat([normal_df.iloc[:372], normal_df.iloc[780:-408]]) # 나머지
dev_normal = normal_df.iloc[372:780]
test_normal = normal_df.iloc[-408:]

In [13]:
train_error = pd.concat([error_df.iloc[:60], error_df.iloc[96:-36]]) # 나머지
dev_error = error_df.iloc[60:96] # 3월~4월
test_error = error_df.iloc[-36:] # 7월

In [302]:
# normal_df.to_json('normal_df.json', orient='records')

In [303]:
# error_df.to_json('error_df.json', orient='records')

### hist 데이터 기준 날짜 인덱스 붙이기

In [26]:
import pandas as pd

def groupping_data(df_data):
    df_data['created_at_datetime']=pd.to_datetime(df_data['created_at_datetime'])
    df_data['new_date'] = (df_data['created_at_datetime'] + pd.Timedelta(days=1)).dt.date
    # df_data.loc[:, 'created_at_datetime'] = pd.to_datetime(df_data['created_at_datetime'])
    # df_data.loc[:, 'new_date'] = (df_data['created_at_datetime'] + pd.Timedelta(days=1)).dt.date
    df = df_data
    
    nv = df['new_date']
    nv = nv.reset_index(drop=True)
    nv = nv.groupby(nv.index // 12).first()
    ni = pd.MultiIndex.from_product([nv, range(12)],names=['date', 'cnt'])

    df.index = ni
    df = df.drop(columns=['created_at_datetime','new_date'])

    return df

In [15]:
train_normal = groupping_data(train_normal)
dev_normal = groupping_data(dev_normal)
test_normal = groupping_data(test_normal)

C:\Users\user\AppData\Local\Temp\ipykernel_13200\1234166630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['created_at_datetime']=pd.to_datetime(df_data['created_at_datetime'])
C:\Users\user\AppData\Local\Temp\ipykernel_13200\1234166630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['new_date'] = (df_data['created_at_datetime'] + pd.Timedelta(days=1)).dt.date
C:\Users\user\AppData\Local\Temp\ipykernel_13200\1234166630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a 

## csv 파일로 저장 후 로드 테스트

In [37]:
def save_df_to_csv(dfs):
    for df in dfs:
        df_name = [name for name, value in globals().items() if value is df][0]
        df.to_csv(f"5528_spec_{df_name}.csv", index=True)

In [38]:
save_df_to_csv([train_normal, dev_normal, test_normal, train_error, test_error, dev_error])

In [2]:
import pandas as pd

In [3]:
train_normal = pd.read_csv("5528_spec_train_normal.csv", header=[0], index_col=[0,1])
test_normal = pd.read_csv("5528_spec_test_normal.csv", header=[0], index_col=[0,1])
dev_normal = pd.read_csv("5528_spec_dev_normal.csv", header=[0], index_col=[0,1])
train_error = pd.read_csv("5528_spec_train_error.csv", header=[0], index_col=[0,1])
test_error = pd.read_csv("5528_spec_test_error.csv", header=[0], index_col=[0,1])
dev_error = pd.read_csv("5528_spec_dev_error.csv", header=[0], index_col=[0,1])

In [4]:
train_normal

created_at                                     spectrum_x_amp  \
date       cnt                                                                  
2024-01-12 0    1704901985  [0.00042537287034367515, 0.0006416402772945482...   
           1    1704909194  [0.0004952475768906472, 0.00046743998424794654...   
           2    1704916397  [0.00024951111178828035, 0.0002839992925661564...   
           3    1704923605  [1.0507872185707666e-05, 6.886010749249397e-05...   
           4    1704930808  [0.0004678164019078649, 0.0013137867109838262,...   
...                    ...                                                ...   
2024-07-17 7    1721109738  [3.794711177547792e-06, 8.360340058540437e-05,...   
           8    1721116946  [0.0017597530997779602, 0.003434518442116838, ...   
           9    1721124149  [0.00015344175072115685, 0.0002784958419638431...   
           10   1721131358  [2.2816951087818676e-05, 0.0002381990981198192...   
           11   1721138561  [8.39292907789931e-05, 0.00013756733161201487,...   

                                                   spectrum_y_amp  \
date       cnt                                                      
2024-01-12 0    [0.00020635396946099019, 0.0003100558571815329...   
           1    [0.00016051611590231604, 0.0001113571030668683...   
           2    [9.14235290461346e-05, 0.00010227322932295558,...   
           3    [0.00015404194614079104, 0.0002930843035366268...   
           4    [0.0012457594683349597, 0.002439545756240494, ...   
...                                                           ...   
2024-07-17 7    [5.158602416061172e-05, 0.0001335283417852297,...   
           8    [0.000855532724403828, 0.0010382324043795453, ...   
           9    [0.00015782670045063344, 0.0002841484978428752...   
           10   [6.2306003242324115e-06, 0.0001890299722825689...   
           11   [0.00011150408848274958, 0.0001686535261631363...   

                                                   spectrum_z_amp  \
date       cnt                                                      
2024-01-12 0    [0.0002770334279662072, 0.0004462952954874889,...   
           1    [0.00029144869690031743, 0.0003112218193772041...   
           2    [0.00018883640812350743, 0.0003188173517410369...   
           3    [0.00039784871799657633, 0.0005350226945562514...   
           4    [0.0007305390559110608, 0.0005454625258034126,...   
...                                                           ...   
2024-07-17 7    [0.00015860936179732686, 0.0002492915112495947...   
           8    [0.00010025577400239155, 0.0006600100497244673...   
           9    [0.00020409381617228793, 0.0004203363820861177...   
           10   [0.00020007279946498342, 0.0002049642939436726...   
           11   [7.352716542191543e-05, 0.00027507572462698884...   

                imbalance_health  
date       cnt                    
2024-01-12 0                 1.0  
           1                 1.0  
           2                 1.0  
           3                 1.0  
           4                 1.0  
...                          ...  
2024-07-17 7                 1.0  
           8                 1.0  
           9                 1.0  
           10                1.0  
           11                1.0  

[1332 rows x 5 columns]

In [5]:
import csv_to_tensor as ct
train_normal = ct.PostProcessing("5528_spec_train_normal.csv")
test_normal = ct.PostProcessing("5528_spec_test_normal.csv")
dev_normal = ct.PostProcessing("5528_spec_dev_normal.csv")
train_error = ct.PostProcessing("5528_spec_train_error.csv")
test_error = ct.PostProcessing("5528_spec_test_error.csv")
dev_error = ct.PostProcessing("5528_spec_dev_error.csv")

In [6]:
import pandas as pd
import torch
from itertools import chain

def PostProcessing_Reg(file_name, cols=['spectrum_x_amp', 'spectrum_y_amp', 'spectrum_z_amp']):
    """  Index=True 로 저장된 csv 파일을 전처리하여 stack 된 tensor 반환

    file_name : 파일 경로
    cols : 컬럼 리스트, 기본값) ['spectrum_x_amp', 'spectrum_y_amp', 'spectrum_z_amp']
    """
    # csv 파일 로드
    df = pd.read_csv(file_name, header=[0], index_col=[0, 1])
    
    # 각 행의 리스트를 병합, 첫번째 데이터는 버림
    for col in cols:
        df[col] = df[col].apply(lambda x: [float(j) for j in x.replace('[', '').replace(']', '').split(',')][1:])
    
    # 각 컬럼의 데이터를 병합
    new_df = df[cols].apply(lambda x: list(chain.from_iterable(x)), axis=1)

    # 라벨링용 imbalance_health 처리
    imb = df['imbalance_health']
    imb = imb.reset_index(drop=True)
    imb = imb.groupby(imb.index // 12).first()
    
    # 그룹별로 데이터를 모아서 텐서로 변환하고 스택
    tensors = [torch.tensor(item) for item in new_df.groupby(level=0).apply(lambda x: [item for sublist in x for item in sublist])]
    
    # 텐서들을 스택
    result = torch.stack(tensors)

    return result, imb


In [11]:
train_normal, im =PostProcessing_Reg("5528_spec_train_normal.csv")
train_error, im = PostProcessing_Reg("5528_spec_train_error.csv")

In [12]:
train_error, im

(tensor([[1.5478e-04, 1.2810e-04, 8.3613e-05,  ..., 3.9515e-04, 1.9020e-04,
          2.5388e-05],
         [1.1565e-04, 1.6415e-04, 1.0496e-04,  ..., 6.5325e-04, 4.9004e-04,
          1.0035e-04],
         [2.0032e-04, 1.5923e-04, 1.6927e-04,  ..., 5.9720e-04, 7.9742e-04,
          8.2897e-04],
         ...,
         [1.1860e-04, 7.7795e-05, 1.0226e-04,  ..., 2.1402e-03, 1.8804e-03,
          1.6231e-03],
         [9.9313e-05, 1.4456e-04, 2.2368e-04,  ..., 5.0263e-04, 5.0777e-04,
          5.9311e-04],
         [3.7712e-04, 4.5022e-04, 4.5768e-04,  ..., 3.3496e-04, 3.8365e-04,
          1.2905e-04]]),
 0    0.924009
 1    0.878773
 2    0.939328
 3    0.888119
 4    0.857143
 5    0.639635
 6    0.850000
 7    0.836673
 8    0.925000
 9    0.950000
 Name: imbalance_health, dtype: float64)

In [13]:
len(train_error)

10

In [14]:
len(im)

10

## 라벨링 -> 이진분류

In [21]:
import torch
def Labeling_data_cls(normal_data, error_data):
    # 데이터 레이블 생성
    normal_labels = torch.ones(normal_data.size(0), dtype=torch.long)  # 정상 데이터 레이블 (1)
    error_labels = torch.zeros(error_data.size(0), dtype=torch.long)  # 비정상 데이터 레이블 (0)
    
    # 데이터 결합
    combined_data = torch.cat((normal_data, error_data), dim=0)
    combined_labels = torch.cat((normal_labels, error_labels), dim=0)

    return combined_data, combined_labels

## 라벨링 -> 회귀

In [15]:
import torch
def Labeling_data_reg(normal_data, error_data, imb):
    # 데이터 레이블 생성
    normal_labels = torch.ones(normal_data.size(0), dtype=torch.long)  # 정상 데이터 레이블 (1)
    error_labels = torch.tensor(imb.values, dtype=torch.float32)  # 비정상 데이터 레이블 (0)
    
    # 데이터 결합
    combined_data = torch.cat((normal_data, error_data), dim=0)
    combined_labels = torch.cat((normal_labels, error_labels), dim=0)

    return combined_data, combined_labels

In [16]:
combined_data, label = Labeling_data_reg(train_normal, train_error, im)

In [17]:
combined_data

tensor([[6.4164e-04, 6.9671e-04, 4.5043e-04,  ..., 3.1335e-05, 1.6753e-04,
         2.6539e-04],
        [5.7142e-05, 5.6895e-05, 3.8271e-05,  ..., 1.9522e-04, 3.5067e-04,
         3.5810e-04],
        [2.7202e-04, 4.8636e-04, 5.3246e-04,  ..., 5.2438e-03, 1.7740e-03,
         1.1917e-03],
        ...,
        [1.1860e-04, 7.7795e-05, 1.0226e-04,  ..., 2.1402e-03, 1.8804e-03,
         1.6231e-03],
        [9.9313e-05, 1.4456e-04, 2.2368e-04,  ..., 5.0263e-04, 5.0777e-04,
         5.9311e-04],
        [3.7712e-04, 4.5022e-04, 4.5768e-04,  ..., 3.3496e-04, 3.8365e-04,
         1.2905e-04]])

In [18]:
label

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.9240, 

In [23]:
train_data, train_label = Labeling_data_cls(train_normal, train_error)
test_data, test_label = Labeling_data_cls(test_normal, test_error)
dev_data, dev_label = Labeling_data_cls(dev_normal, dev_error)

In [18]:
train_data

tensor([[6.4164e-04, 6.9671e-04, 4.5043e-04,  ..., 3.1335e-05, 1.6753e-04,
         2.6539e-04],
        [5.7142e-05, 5.6895e-05, 3.8271e-05,  ..., 1.9522e-04, 3.5067e-04,
         3.5810e-04],
        [2.7202e-04, 4.8636e-04, 5.3246e-04,  ..., 5.2438e-03, 1.7740e-03,
         1.1917e-03],
        ...,
        [1.1860e-04, 7.7795e-05, 1.0226e-04,  ..., 2.1402e-03, 1.8804e-03,
         1.6231e-03],
        [9.9313e-05, 1.4456e-04, 2.2368e-04,  ..., 5.0263e-04, 5.0777e-04,
         5.9311e-04],
        [3.7712e-04, 4.5022e-04, 4.5768e-04,  ..., 3.3496e-04, 3.8365e-04,
         1.2905e-04]])

In [19]:
train_label

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

In [1]:
import ttt

In [4]:
result = ttt.PostProcessing('data/5528_droped_data.csv', model_type='reg')

C:\MotorPM Project\MotorPMData\ttt.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: [float(j) for j in x][1:])
C:\MotorPM Project\MotorPMData\ttt.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: [float(j) for j in x][1:])
C:\MotorPM Project\MotorPMData\ttt.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [5]:
result[0][1]

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.9240, 

In [6]:
result2 = ttt.PostProcessing('data/5528_droped_data.csv', model_type='cls')

C:\MotorPM Project\MotorPMData\ttt.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: [float(j) for j in x][1:])
C:\MotorPM Project\MotorPMData\ttt.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: [float(j) for j in x][1:])
C:\MotorPM Project\MotorPMData\ttt.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [7]:
result2[0][1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])